# 项目：整理Netflix电影演员评分数据

## 分析目标

此数据分析的目的是，整理不同流派影视作品，比如喜剧片、动作片、科幻片中，各演员出演作品的平均IMDB评分，从而挖掘出各个流派中的高评分作品演员。

本实战项目的目的在于练习整理数据，从而得到可供下一步分析的数据。

## 简介

原始数据集记录了截止至2022年7月美国地区可观看的所有Netflix电视剧及电影数据。数据集包含两个数据表：`titles.csv`和`credits.csv`。

`titles.csv`包含电影及电视剧相关信息，包括影视作品ID、标题、类型、描述、流派、IMDB（一个国外的在线评分网站）评分，等等。`credits.csv`包含超过7万名出现在Netflix影视作品的导演及演员信息，包括名字、影视作品ID、人物名、演职员类型（导演/演员）等。

`titles.csv`每列的含义如下：
- id：影视作品ID。
- title：影视作品标题。
- show_type：作品类型，电视节目或电影。
- description：简短描述。
- release_year：发布年份。
- age_certification：适龄认证。
- runtime：每集电视剧或电影的长度。
- genres：流派类型列表。
- production_countries：出品国家列表。
- seasons：如果是电视剧，则是季数。
- imdb_id：IMDB的ID。
- imdb_score：IMDB的评分。
- imdb_votes：IMDB的投票数。
- tmdb_popularity：TMDB的流行度。
- tmdb_score：TMDB的评分。

`credits.csv`每列的含义如下：
- person_ID：演职员ID。
- id：参与的影视作品ID。
- name：姓名。
- character_name：角色姓名。
- role：演职员类型，演员或导演。

In [1]:
import pandas as pd

# 1. 读取数据

In [5]:
# 使用 pandas.read_csv()方法分别读取所需文件,就是把.csv文件解析为DataFrame对象 赋值给变量
original_titles = pd.read_csv("./titles.csv")
original_credits = pd.read_csv("./credits.csv")

# 2.1. 评估数据 titles.csv

#### 评估数据整洁度
多用 .head() .tail() .sample() 查看表开头，表结尾，表随机部分行数。

In [10]:
# 1: 使用 .sample() 函数随机抽取数据进行查看
original_titles.sample(5)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
4223,tm837648,Blanche Gardin - Bonne nuit Blanche,MOVIE,"Blanche offers us her new stand-up, creation 2...",2019,NaN,95,['comedy'],['FR'],NaN,NaN,NaN,NaN,3.751,7.6
2185,tm357820,The Open House,MOVIE,A teenager and his mother find themselves besi...,2018,NaN,94,"['thriller', 'horror']",['US'],NaN,tt7608028,3.3,34156.0,13.150,3.7
2727,tm431928,Rajma Chawal,MOVIE,A father attempts to reconnect with his estran...,2018,NaN,129,"['drama', 'comedy', 'family']",['IN'],NaN,tt6747420,5.7,1853.0,3.305,6.5
2152,ts83457,Cleo & Cuquin,SHOW,Sister and brother Cleo and Cuquin carry out d...,2018,TV-Y,7,"['animation', 'family']",['ES'],2.0,tt7956374,8.1,98.0,2.076,8.3
4779,ts287722,The Upshaws,SHOW,"Bennie Upshaw, the head of a Black working cla...",2021,TV-14,27,"['family', 'comedy']",['US'],2.0,tt10945036,6.8,2589.0,5.205,8.7


数据的结构性问题指不符合`"每列是一个变量，每行是一个观察值，每个单元是一个值"`这三个标准。
    1. `genres：流派类型列表`列不符合 `每个单元是一个值`的标准。

# 评估数据干净程度
多用 .info()方法 获取DataFrame对象概况

In [11]:
# 使用 .info()方法 获取DataFrame对象概况
original_titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5850 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    5850 non-null   object 
 1   title                 5849 non-null   object 
 2   type                  5850 non-null   object 
 3   description           5832 non-null   object 
 4   release_year          5850 non-null   int64  
 5   age_certification     3231 non-null   object 
 6   runtime               5850 non-null   int64  
 7   genres                5850 non-null   object 
 8   production_countries  5850 non-null   object 
 9   seasons               2106 non-null   float64
 10  imdb_id               5447 non-null   object 
 11  imdb_score            5368 non-null   float64
 12  imdb_votes            5352 non-null   float64
 13  tmdb_popularity       5759 non-null   float64
 14  tmdb_score            5539 non-null   float64
dtypes: float64(5), int64(

1. 从输出结果来看，original_titles最大观察值有5850条，其中`title:影视作品标题`,`description:描述`,`age_certification:适龄认证`,`seasons:电视剧`,`imdb_id:IMDB的ID`,`imdb_score:IMDB的评分`,`imdb_votes:IMDB的投票数`,`tmdb_popularity:TMDB的流行度`,`tmdb_score:TMDB评分`都有缺失值。
2. `release_year:发布年份`应该是: 日期类型`pandas.to_datetime`。
3. `imdb_votes:IMDB的投票数`应该是`int64`。

#### 评估数据缺失值
根据分析目标：`整理不同流派影视作品，比如喜剧片、动作片、科幻片中，各演员出演作品的平均IMDB评分，从而挖掘出各个流派中的高评分作品演员。`
主要是看`genres:流派类型列表`，`tmdb_score:TMDB评分`的缺失值。

In [16]:
# 查看缺失值 .isnull()方法
original_titles[original_titles["imdb_score"].isnull()]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
75,tm132164,Bill Hicks: Sane Man,MOVIE,Sane Man was filmed before Bill recorded ‘Dang...,1989,R,80,['comedy'],['US'],NaN,NaN,NaN,NaN,3.377,7.5
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1991,TV-G,18,"['documentation', 'family', 'reality']",['JP'],12.0,NaN,NaN,NaN,7.730,7.8
177,ts224786,Escalona,SHOW,"The improbable real life of Rafael Escalona, w...",1991,TV-MA,44,['drama'],['CO'],1.0,NaN,NaN,NaN,7.352,7.6
180,tm8687,Sam Kinison: Family Entertainment Hour,MOVIE,Get ready to scream with laughter! Sam Kinison...,1991,NaN,49,['comedy'],['US'],NaN,NaN,NaN,NaN,1.840,6.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5810,tm1225897,Social Man,MOVIE,Two competitive social media Influencers go he...,2021,NaN,96,"['comedy', 'drama']",[],NaN,tt20198164,NaN,NaN,NaN,NaN
5833,ts307884,HQ Barbers,SHOW,When a family run barber shop in the heart of ...,2021,TV-14,24,['comedy'],['NG'],1.0,NaN,NaN,NaN,0.840,NaN
5840,tm1216735,Sun of the Soil,MOVIE,"In 14th-century Mali, an ambitious young royal...",2022,NaN,26,[],[],NaN,NaN,NaN,NaN,1.179,7.0
5844,tm1074617,Bling Empire - The Afterparty,MOVIE,"The stars of ""Bling Empire"" discuss the show's...",2021,NaN,35,[],['US'],NaN,NaN,NaN,NaN,NaN,NaN


1. `tmdb_score:TMDB评分`共有482条缺失值，不能作为分析目标的判断依据。

#### 评估重复数据
多用 .duplicated()方法评估重复数据
`genres:流派类型列表`，`tmdb_score:TMDB评分`,都是可以完全重复的。可以尝试查看`id:影视作品`是否有重复值。

In [22]:
# 查看整行Series对象是否有重复值
original_titles[(original_titles.duplicated() == True)]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score


In [25]:
# 查看`id:影视作品`是否有重复值。
original_titles[(original_titles["id"].duplicated() == True)]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score


#### 评估不一致数据
多用 .value_counts()方法评估不一致数据
主要查看的 `production_countries:出品国家列表`不一致数据。

In [26]:
# 查看`production_countries:出品国家列表`不一致数据
original_titles["production_countries"].value_counts()

production_countries
['US']                1959
['IN']                 599
['JP']                 264
[]                     229
['KR']                 222
                      ... 
['CZ', 'GB', 'US']       1
['US', 'AU', 'GB']       1
['HU']                   1
['IQ', 'GB']             1
['PH', 'FO']             1
Name: count, Length: 452, dtype: int64

#### 评估无效或错误数据
可以尝试用 .describe()方法查看DataFrame对象的统计信息

In [28]:
# 查看 original_titles 的统计信息
original_titles.describe()

,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
count,5850.000000,5850.000000,2106.000000,5368.000000,5.352000e+03,5759.000000,5539.000000
mean,2016.417094,76.888889,2.162868,6.510861,2.343938e+04,22.637925,6.829175
std,6.937726,39.002509,2.689041,1.163826,9.582047e+04,81.680263,1.170391
min,1945.000000,0.000000,1.000000,1.500000,5.000000e+00,0.009442,0.500000
25%,2016.000000,44.000000,1.000000,5.800000,5.167500e+02,2.728500,6.100000
50%,2018.000000,83.000000,1.000000,6.600000,2.233500e+03,6.821000,6.900000
75%,2020.000000,104.000000,2.000000,7.300000,9.494000e+03,16.590000,7.537500
max,2022.000000,240.000000,42.000000,9.600000,2.294231e+06,2274.044000,10.000000


# 数据清洗
根据 评估数据 得到的需要清洗的结果:
1. `genres:流派类型列表`需要进行拆分。
2. `production_countries:出品国家`需要进行值转换
3. `release_year:发布年份`应该是: 日期类型`pandas.to_datetime`。
4. `imdb_votes:IMDB的投票数`应该是`int64`。
5. `tmdb_score:TMDB评分`缺失值需要删除。

In [29]:
# 在清理数据之前,对原始数据做好备份
cleaned_titles = original_titles.loc[:,:]

1. `genres:流派类型列表`需要进行拆分
虽然表示形式是列表，但其实际类型并非字符串列表，而是字符串，难以进行拆分。我们可以再次利用`eval`函数进行类型转换，并检查转换后确实是列表类型。

In [ ]:
# 1. `genres:流派类型列表`需要进行拆分

#先提取出任意一行进行观察
cleaned_titles["genres"][1] #输出 "['drama', 'crime']" 

#查看其数据类型
print(type(cleaned_titles["genres"][1])) #输出 <class 'str'>

#输出可以看出,其并不是列表,是字符串。需要把 cleaned_titles["genres"] 转换为列表形式在赋值给 cleaned_titles["genres"]
cleaned_titles["genres"] = cleaned_titles["genres"].apply(lambda x:eval(x))

#再次查看数据类型
print(type(cleaned_titles["genres"][1])) #输出 <class 'list'>

In [53]:
# 1. `genres:流派类型列表`需要进行拆分 ----续
#进行拆分 .explode()方法对行进行拆分
cleaned_titles = cleaned_titles.explode("genres")

2. `production_countries:出品国家`需要进行值转换
相同的问题:
虽然表示形式是列表，但其实际类型并非字符串列表，而是字符串，难以进行拆分。我们可以再次利用`eval`函数进行类型转换，并检查转换后确实是列表类型。

In [ ]:
# 2. `production_countries:出品国家`需要进行值转换

#提取任意一行数据进行观察
cleaned_titles["production_countries"][1]

#查看其数据类型
print(type(cleaned_titles["production_countries"][1])) #输出：<class 'pandas.core.series.Series'>

#实际类型并不是列表 需要用到 eval()函数进行转换
cleaned_titles["production_countries"] = cleaned_titles["production_countries"].apply(lambda x:eval(x))

In [77]:
# 2. `production_countries:出品国家`需要进行值转换 ----续
#进行拆分 .explode()方法对行下 列表值拆分
cleaned_titles= cleaned_titles.explode("production_countries")

#查看值,转换成功
cleaned_titles["production_countries"]

0        US
1        US
1        US
2        US
2        US
       ... 
5847     CO
5848     US
5849    NaN
5849    NaN
5849    NaN
Name: production_countries, Length: 17818, dtype: object

3. `release_year:发布年份`应该是: 日期类型`pandas.to_datetime`。

In [89]:
cleaned_titles["release_year"] = pd.to_datetime(cleaned_titles["release_year"])

4. `imdb_votes:IMDB的投票数`应该是`int64`。
`imdb_votes:IMDB的投票数`投票数从缺失值NaN 变为 .fillna(0) 不会影响其最终判断

In [90]:
cleaned_titles["imdb_votes"] = cleaned_titles["imdb_votes"].fillna(0).astype("int64")

5. `tmdb_score:TMDB评分`缺失值需要删除。
流程: 提取出带删除的所有行(.index)并复制给一个临时变量。然后使用 .drop()方法删除

方法二: `.dropna()方法`删除tmdb_score列下,所有 tmdb_score 为 NaN的数据行(axis=0)
cleaned_titles.dropna(subset=["tmdb_score"],axis=0)

In [96]:
# 提取出带删除的所有行(.index)并复制给一个临时变量
_ = cleaned_titles[cleaned_titles["tmdb_score"].isnull()].index
# 使用 .drop()方法删除并且重新赋值
cleaned_titles = cleaned_titles.drop(_)

# 2.2. 评估数据 credits.csv

#### 评估数据整洁度

In [98]:
# .sample()方法随机查看
original_credits.sample(5)

,person_id,id,name,character,role
31313,121988,ts76313,Ming-Zhu Hii,NaN,ACTOR
52623,8563,ts106141,Giselle Itié,Eva,ACTOR
35114,1609093,tm285295,Constance Freeman,Hostess,ACTOR
2468,469828,tm116655,William James Stiggers Jr.,Subway Rapper,ACTOR
67136,174540,ts283080,Eugenio Siller,Jose Maria,ACTOR


#### 评估数据干净程度

In [99]:
original_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77801 entries, 0 to 77800
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  77801 non-null  int64 
 1   id         77801 non-null  object
 2   name       77801 non-null  object
 3   character  68029 non-null  object
 4   role       77801 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.0+ MB


original_credits对象最大观察值 77801条。`character_name：角色姓名`包含缺失值。
1. `character`应该替换为`character_name：角色姓名`。

#### 评估数据缺失值

In [101]:
original_credits[original_credits["character"].isnull()]

,person_id,id,name,character,role
36,3308,tm84618,Martin Scorsese,NaN,DIRECTOR
59,17727,tm154986,John Boorman,NaN,DIRECTOR
106,11475,tm127384,Terry Jones,NaN,DIRECTOR
107,11473,tm127384,Terry Gilliam,NaN,DIRECTOR
162,1063,tm120801,Robert Aldrich,NaN,DIRECTOR
...,...,...,...,...,...
77776,2363022,tm1097142,Mohamed El-Arkan,NaN,ACTOR
77777,1827884,tm1097142,Mohamed Bakir,NaN,DIRECTOR
77783,678884,tm1014599,Segun Arinze,NaN,ACTOR
77789,1962840,tm1014599,Seyi Babatope,NaN,DIRECTOR


`character_name：角色姓名`为空是不影响`name：姓名`的评分标准的。
但是有点需要注意:
    `role：演职员类型，演员或导演` 我们需要将 role 为 DIRECTOR 的删除。

In [123]:
# 赛选出 `role：演职员类型，演员或导演` 为导演的类型。
original_credits[original_credits["role"] == "DIRECTOR"]

,person_id,id,name,character,role
36,3308,tm84618,Martin Scorsese,NaN,DIRECTOR
59,17727,tm154986,John Boorman,NaN,DIRECTOR
106,11475,tm127384,Terry Jones,NaN,DIRECTOR
107,11473,tm127384,Terry Gilliam,NaN,DIRECTOR
162,1063,tm120801,Robert Aldrich,NaN,DIRECTOR
...,...,...,...,...,...
77745,1785287,tm1216735,Joe Penney,NaN,DIRECTOR
77761,753271,tm985215,Barry Gonzalez,NaN,DIRECTOR
77777,1827884,tm1097142,Mohamed Bakir,NaN,DIRECTOR
77789,1962840,tm1014599,Seyi Babatope,NaN,DIRECTOR


In [114]:
# copy一份用于清洗数据和保存数据
cleaned_credits = original_credits.loc[:,:]

# 保存数据

In [119]:
# 保存数据,默认不保存位置索引
cleaned_titles.to_csv(r"/Users/rong/Desktop/数据整理项目/titles_cleaned.csv.csv",index=False)
cleaned_credits.to_csv(r"/Users/rong/Desktop/数据整理项目/credits_cleaned.csv",index=False)

# 整理数据
现在我们需要把两张表的数据通过某个变量整合在一起，这里`titles.csv`和 `titles.csv`都有`id：影视作品ID`列。
我们可以通过`id：影视作品ID`作为键进行连接,链接方式选择交集。表示在两表中都存在。

In [161]:
# pd.merge()函数进行合并
credits_with_titles = pd.merge(cleaned_titles,cleaned_credits,on="id",how="inner")

合并后,就能知道各个演员职员的具体信息了。

In [162]:
credits_with_titles

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,person_id,name,character,role
0,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1970-01-01 00:00:00.000001976,R,114,drama,US,NaN,tt0075314,8.2,808582,40.965,8.179,3748,Robert De Niro,Travis Bickle,ACTOR
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1970-01-01 00:00:00.000001976,R,114,drama,US,NaN,tt0075314,8.2,808582,40.965,8.179,14658,Jodie Foster,Iris Steensma,ACTOR
2,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1970-01-01 00:00:00.000001976,R,114,drama,US,NaN,tt0075314,8.2,808582,40.965,8.179,7064,Albert Brooks,Tom,ACTOR
3,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1970-01-01 00:00:00.000001976,R,114,drama,US,NaN,tt0075314,8.2,808582,40.965,8.179,3739,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1970-01-01 00:00:00.000001976,R,114,drama,US,NaN,tt0075314,8.2,808582,40.965,8.179,48933,Cybill Shepherd,Betsy,ACTOR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281841,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,1970-01-01 00:00:00.000002021,NaN,90,comedy,CO,NaN,tt14585902,3.8,68,26.005,6.300,736339,Adelaida Buscato,María Paz,ACTOR
281842,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,1970-01-01 00:00:00.000002021,NaN,90,comedy,CO,NaN,tt14585902,3.8,68,26.005,6.300,399499,Luz Stella Luengas,Karen Bayona,ACTOR
281843,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,1970-01-01 00:00:00.000002021,NaN,90,comedy,CO,NaN,tt14585902,3.8,68,26.005,6.300,373198,Inés Prieto,Fanny,ACTOR
281844,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,1970-01-01 00:00:00.000002021,NaN,90,comedy,CO,NaN,tt14585902,3.8,68,26.005,6.300,378132,Isabel Gaona,Cacica,ACTOR


In [163]:
# 使用 .query()筛选出 role 为导演的信息
ACTOR_with_titles = credits_with_titles.query('role == "ACTOR"')

为了挖掘出各个流派中的高IMDB评分作品演员，我们需要先根据流派和演员进行分组。
对演员进行分组的时候，选择的是用 'person_id'而不是'name'变量，原因是名字容易出现错拼或者重名的情况，演职员'ID'会比演员姓名更加准确地反映是哪位演员。

In [164]:
groupby_genres_and_person_id = ACTOR_with_titles.groupby(["genres","person_id"])

分组后，我们只需要对'imdb_score'的值进行聚合计算，因此只提取'imdb_score'变量，然后调用'mean'。来计算各个流派影视作品中，每位参演作品的IMDB评分。

In [165]:
imdb_score_groupby_genres_and_person_id = groupby_genres_and_person_id["imdb_score"].mean()

In [166]:
imdb_score_groupby_genres_and_person_id

genres   person_id
action   45           5.0
         48           5.4
         51           6.4
         53           6.8
         54           5.3
                     ... 
western  2353339      6.9
         2370848      6.1
         2398539      3.8
         2406218      6.0
         2408082      7.3
Name: imdb_score, Length: 171549, dtype: float64

可以调用'reset_index',对层次化索引进行重置,得到更加规整的DataFrame对象。

In [167]:
_ = imdb_score_groupby_genres_and_person_id.reset_index()
_

,genres,person_id,imdb_score
0,action,45,5.0
1,action,48,5.4
2,action,51,6.4
3,action,53,6.8
4,action,54,5.3
...,...,...,...
171544,western,2353339,6.9
171545,western,2370848,6.1
171546,western,2398539,3.8
171547,western,2406218,6.0


找出各个流派里演员作品最高的平均分是多少？最高评分对应的演员名字是什么？
需要对上面的 '_'对象进行分组，再次用‘genres’进行分组,然后提取'imdb_score'变量,计算其最大值。

为什么不在进行 _.groupby("genres")分组 的时候把 person_id 也提取出来呢？
因为分组后进行的聚合操作 会被应用在进行聚合操作列 ‘如：("genres")["imdb_score"]的["imdb_score"]’ 字段的所有数据上。

In [168]:
# 查看各个 genres:流派里面  imdb_score 的最大值
_01 = _.groupby("genres")["imdb_score"].max()
_01

genres
action           9.3
animation        9.3
comedy           9.2
crime            9.5
documentation    9.1
drama            9.5
european         8.9
family           9.3
fantasy          9.3
history          9.1
horror           9.0
music            8.8
reality          8.9
romance          9.2
scifi            9.3
sport            9.1
thriller         9.5
war              8.8
western          8.9
Name: imdb_score, dtype: float64

现在已经求出了各个流派平均分的最大值,但是我们不知道这些平均分的最大值对应的名字。
所以我们需要让 _01 与 _ 继续进行合并,主键就是 genres

In [169]:
# _ 包含了所有流派的所有平均分
# _01 包含了所有流派的最高平均分
# 目标是一眼看出流派下最高的平均分
_02 = pd.merge(_,_01,on=["genres","imdb_score"],how="inner")
_02

,genres,person_id,imdb_score
0,action,1303,9.3
1,action,12790,9.3
2,action,21033,9.3
3,action,86591,9.3
4,action,336830,9.3
...,...,...,...
131,war,826547,8.8
132,western,22311,8.9
133,western,28166,8.9
134,western,28180,8.9


In [172]:
# 查看最高平均评分演员ID的名字。把演员ID和名字提取出来单独保存，继续连接。
name_id = credits_with_titles.loc[:,["person_id","name"]]

pd.merge(name_id,_02,on=["person_id"])

,person_id,name,genres,imdb_score
0,22311,Koichi Yamadera,western,8.9
1,22311,Koichi Yamadera,western,8.9
2,22311,Koichi Yamadera,western,8.9
3,22311,Koichi Yamadera,western,8.9
4,22311,Koichi Yamadera,western,8.9
...,...,...,...,...
1411,28024,Dante Basco,scifi,9.3
1412,2150,Talulah Riley,music,8.8
1413,129032,Virgile Bramly,music,8.8
1414,129032,Virgile Bramly,music,8.8
